# Astronomy with Python

The aim of this notebook is to cover some features of the Astropy package and introduce some other packages from outside the standard library that might be useful. If you're working on your own machine, you will have to install all these packages yourself. See the SciencePackages notebook for more details.

In [ ]:
# RUN THIS FIRST
import numpy as np
import matplotlib.pyplot as pl
%matplotlib inline

datadir = '/data/pyintro/'

## Astropy

[`astropy`](http://www.astropy.org/) is a large third-party Python package written as a collaborative effort among many astronomers. It provides a lot of functions useful in many astronomical tasks, including things like handling spherical coordinates in different systems and reading arrays of data in common formats (including astro-specific things like FITS headers that define exotic coordinate systems, and VOTables). 

`astropy` is in continuous and active development, although the core parts are now quite stable. Don't trust `astropy` to be absolutely bullet-proof for anything you really care about, but do use it to avoid re-inventing the wheel. 

Also, if you use it for work that goes into a paper, you should mention it in the Acknowledgements. Strictly speaking this should also go for `numpy`, `scipy`, `matplotlib` and anything else you use that was written by someone else. This is still not as widespread as it should be.

In [ ]:
import astropy

## Tables

Astropy's `Table` class is a very useful for dealing with most astronomical data in tabular form, particularly that read from FITS files, a common 'structured binary' format developed for astronomy and widely used for most observational data. Full documentation [here](http://docs.astropy.org/en/stable/table/), including a 'Getting Started' section.

As an example, let's say we have some data represented as a `numpy` `recarray` object:

In [ ]:
# Set up some data as a numpy record array
# (data from Hogg, Bovy & Lang 2010)
line_data = np.array([(1, 201.0, 592.0, 9.0, 61.0, -0.84),
       (2, 244.0, 401.0, 4.0, 25.0, 0.31),
       (3, 47.0, 583.0, 11.0, 38.0, 0.64),
       (4, 287.0, 402.0, 7.0, 15.0, -0.27),
       (5, 203.0, 495.0, 5.0, 21.0, -0.33),
       (6, 58.0, 173.0, 9.0, 15.0, 0.67),
       (7, 210.0, 479.0, 4.0, 27.0, -0.02),
       (8, 202.0, 504.0, 4.0, 14.0, -0.05),
       (9, 198.0, 510.0, 11.0, 30.0, -0.84),
       (10, 158.0, 416.0, 7.0, 16.0, -0.69),
       (11, 165.0, 393.0, 5.0, 14.0, 0.3),
       (12, 201.0, 442.0, 5.0, 25.0, -0.46),
       (13, 157.0, 317.0, 5.0, 52.0, -0.03),
       (14, 131.0, 311.0, 6.0, 16.0, 0.5),
       (15, 166.0, 400.0, 6.0, 34.0, 0.73),
       (16, 160.0, 337.0, 5.0, 31.0, -0.52),
       (17, 186.0, 423.0, 9.0, 42.0, 0.9),
       (18, 125.0, 334.0, 8.0, 26.0, 0.4),
       (19, 218.0, 533.0, 6.0, 16.0, -0.78),
       (20, 146.0, 344.0, 5.0, 22.0, -0.56)], 
      dtype=[('ID', '<i4'), ('x', '<f8'), ('y', '<f8'), ('sigma_x', '<f8'), ('sigma_y', '<f8'), ('rho_xy', '<f8')])

It's easy to turn this into an astropy table:

In [ ]:
# Tables; reading and writing FITS
from astropy.table import Table, Column

t = Table(line_data)
print(t)

It's easy to save tables as FITS files and read FITS files directly into tables.

In [ ]:
t.write('line_data.fits',overwrite=True)
t = Table.read('line_data.fits')

It's also very easy to read CSV files directly to Tables.

For more complicated tasks with FITS, there is the `astropy.io.fits` module, which is well explained in [this tutorial](http://www.astropy.org/astropy-tutorials/FITS-images.html).

## Sky coordinates

To demonstrate the functions for spherical coordinate conversions in `astropy`, we're going to use an N-body model of the [Sagittarius stellar stream](https://en.wikipedia.org/wiki/Sagittarius_Stream) created by Law & Majewski (2010)  [(data borrowed from the Gaia Challenge workshop, courtesy of A. Price-Whelan)](http://astrowiki.ph.surrey.ac.uk/dokuwiki/doku.php?id=tests:streams:challenges#the_sagittarius_stream). I've already included this in the notebook data directory as the file `all_particles.txt`. The next cell reads this data into an astropy Table, via `numpy.genfromtxt`, and prints some information.

In [ ]:
d = Table(np.genfromtxt('%s/all_particles.txt'%(datadir),names=True,delimiter=','))
print(len(d))
print(d.colnames)
print(d[0:5])

# Galactic longitude and latitude
l,b = d['l'],d['b']

print('')
print('Ranges of galactic coordinates:')
print('{} < l < {}'.format(l.min(),l.max()))
print('{} < b < {}'.format(b.min(),b.max()))

In case you're interested, the data in this table is:
    - Cartesian xyz position of each particle in kiloparsecs, with the origin at the Galactic centre
    - Cartesian velocity of each particle (3 components, in km/s)
    - Galactic longitude l (degrees)
    - Galactic latitude b (degrees)
    - Distance of the particle (from the Sun?)
    - The (Heliocentric?) radial velocity (in km/s)
    - The apparent angular proper motion of the particle in l and b (micro-arcseconds per year)
    
We'll only be using the xyz and lb coordinates. Let's plot these in an un-exciting way so we know roughly what the data look like:

In [ ]:
pl.figure()
pl.scatter(l,b,c='k',s=1,edgecolor='None',label='Sgr stream stars')
pl.scatter([180],[0],c='b',label='Galactic anti-centre',zorder=10)
pl.legend(loc='upper left',scatterpoints=1,markerscale=2)
pl.xlabel('$l$ (deg)')
pl.ylabel('$b$ (deg)')
pl.xlim(0,360)
pl.ylim(-90,90)

pl.figure(figsize=(4,4))
pl.scatter(d['x'],d['y'],c='k',alpha=0.2,s=1,edgecolor='None',label=None)
pl.scatter([0],[0],c='r',label='Galactic centre')
pl.xlabel('X (kpc)')
pl.ylabel('Y (kpc)')
pl.legend(loc='upper left',scatterpoints=1,markerscale=2);

The following example shows how to convert between the galactic coordinates we have, and the equatorial (RA and DEC) coordinates that would be used to point a telescope. You can read more about `astropy` coordinate conversions in the tutorial [here](http://docs.astropy.org/en/stable/coordinates/) and the functions of the `SkyCoord` object [here](http://docs.astropy.org/en/stable/coordinates/skycoord.html)

In [ ]:
import astropy.coordinates as c
import astropy.units as u

# Make a 'galactic coordinates' object using the data we have. 
# Note that we need to tell astropy what the units of our coordinates
# are using the astropy.units package. We have coordinates in degrees,
# so we multiply our numbers for l and b by astropy.units.deg:
gal_coords = c.SkyCoord(l*u.deg,b*u.deg,'galactic')

# Get the equatorial representation of our galactic coordinates. This is 
# a 'property' of our galactic coordinates object.
# (ICRS is an international standard equatorial coordinate system)
equ_coords = gal_coords.icrs

# Do the same thing for the galactic coordinates of the Galactic centre (l,b) = (0,0)
# and anti-centre (l,b) = (180,0)
gal_centre_equ = c.SkyCoord(0*u.deg,0*u.deg,'galactic').icrs
gal_anti_equ   = c.SkyCoord(180*u.deg,0*u.deg,'galactic').icrs

# Plot the points in RA and DEC. Note we get these as methods of the coordinates object.
pl.figure()
pl.scatter(equ_coords.ra,equ_coords.dec,c='k',s=1,edgecolor='None',label='Sgr stream stars')

# Plot the galactic centre and anti-centre
pl.scatter(gal_centre_equ.ra,gal_centre_equ.dec,s=32,c='r',zorder=10,label='Centre')
pl.scatter(gal_anti_equ.ra,gal_anti_equ.dec,s=32,c='c',zorder=10,label='Anti-centre')

pl.xlabel('RA (deg)')
pl.ylabel('DEC (deg)')

pl.legend(loc='lower left',scatterpoints=1,markerscale=2)
pl.xlim(0,360)
pl.ylim(-90,90);

The `SkyCoord` object has loads of functions. For example:

In [ ]:
# What constellation is the centre of the Galaxy in?
gal_centre_equ.get_constellation()

In [ ]:
# And the anticentre?
gal_anti_equ.get_constellation()

Finally, this example shows how to plot the equatorial data in an Aitoff projection. Note we have to ensure that the longitude is in radians from -pi to pi, and the latitude is in radians from 0 to +pi.

In [ ]:
pl.figure(figsize=(8,4.2))
pl.subplot(111, projection="aitoff")
pl.grid(True)
pl.scatter(equ_coords.ra.wrap_at(180 * u.deg).radian, 
           equ_coords.dec.radian,
           s=1,edgecolor='None',alpha=0.3,c='k');

The next example shows how to draw a line representing the galactic plane on the Aitoff plot above This isn't totally straightforward. To avoid strange wrapping effects, it's necessary to draw the line with `scatter` rather than `plot`.

By the way, Sagittarius is a 'polar' stream.

In [ ]:
pl.figure(figsize=(8,4.2))
pl.subplot(111, projection="aitoff")
pl.grid(True)
pl.scatter(equ_coords.ra.wrap_at(180 * u.deg).radian, 
           equ_coords.dec.radian,
           s=1,edgecolor='None',alpha=0.3,c='k');

plane_l = np.linspace(-np.pi,np.pi,100) 
plane_b = np.repeat(0,len(plane_l))

plane_eq = c.SkyCoord(plane_l*u.radian,plane_b*u.radian,'galactic').fk5

pl.scatter(plane_eq.ra.wrap_at(180.0*u.deg).radian,plane_eq.dec.radian,c='r',marker='.',edgecolor='None')

# Plot the galactic centre and anti-centre
pl.scatter(gal_centre_equ.ra.wrap_at(180.0*u.deg).radian,
           gal_centre_equ.dec.radian,s=32,c='r',zorder=10,label='Centre')

pl.scatter(gal_anti_equ.ra.wrap_at(180*u.deg).radian,
           gal_anti_equ.dec.radian,s=32,c='c',zorder=10,label='Anti-centre');


Another tip about scatter plots: you can pass an array of the same length as your x and y coordinates to the `c=` (`color=`) keyword option. The example below uses this to colour the points in the stream according to their radial velocity (`vr` in the data).

Unlike with `imshow`, to plot a colourbar associated with coloured scatter points requires that the first argument to `colorbar` be the return value of the call `scatter` (assigned to `s` in the example), otherwise you'll get a confusing error about `ScalarMappable`.

In [ ]:
pl.figure(figsize=(8,4.2))
pl.subplot(111, projection="aitoff")
pl.grid(True)
s = pl.scatter(equ_coords.ra.wrap_at(180 * u.deg).radian, 
           equ_coords.dec.radian,
           s=1,edgecolor='None',alpha=0.3,c=d['vr']);

plane_l = np.linspace(-np.pi,np.pi,100) 
plane_b = np.repeat(0,len(plane_l))

plane_eq = c.SkyCoord(plane_l*u.radian,plane_b*u.radian,'galactic').fk5

pl.scatter(plane_eq.ra.wrap_at(180.0*u.deg).radian,
           plane_eq.dec.radian,marker='.',edgecolor='None',c='g')

# Plot the galactic centre and anti-centre
pl.scatter(gal_centre_equ.ra.wrap_at(180.0*u.deg).radian,
           gal_centre_equ.dec.radian,s=32,c='r',zorder=10,label='Centre')

pl.scatter(gal_anti_equ.ra.wrap_at(180*u.deg).radian,
           gal_anti_equ.dec.radian,s=32,c='c',zorder=10,label='Anti-centre')

pl.colorbar(s,label='Radial velocity');

## Astroquery

[Astroquery](http://astroquery.readthedocs.io/en/latest/) is a package that can communicate over the internet with various astronomy databases, including the [Simbad](http://simbad.u-strasbg.fr/guide/simbad.htx) service to turn names of astronomical objects into (RA,DEC) coordinates, and the Sloan Digital Sky Survey (SDSS). SDSS is one of the largest public repositories of astronomical data extracted from a huge number of images of the sky taken by the 2.5m telescope at the Apache Point observatory in New Mexico. 

Astroquery is an 'affiliated package' of the Astropy project, which means it works with Astropy but isn't included in the basic Astropy package. According to  its webpage, it can be installed as follows with `conda`: `conda install -c astropy astroquery`. The `-c astropy` option tells `conda` to use the Astropy project channel to install the `astroquery` package. Channels are online repositories of packages: Anaconda is one such repository, but there are others, including Astropy. Different repositories provide different collections of packages.

Once `astroquery` is installed:

In [ ]:
import astroquery # When you first do this, you might get a harmless warning about the configuration

We also need the `astropy` coordinates and units modules for this example:

In [ ]:
import astropy.coordinates as c
import astropy.units as u

## Example of using astropy and astroquery

NGC 2419 is an unusual Milky Way globular cluster (a sub-galactic clump of several million stars bound tightly together by the gravitational force they exert on one another). This example shows how to use `astroquery` to find the coordinates of the cluster on the sky, and then download a catalog of sources detected near this position in the SDSS images. 

First, use a [Simbad](http://simbad.u-strasbg.fr/simbad/) query to find the coordinates of the cluster.

In [ ]:
import astroquery.simbad
simbad  = astroquery.simbad.Simbad() # get a query manager object
ngc2419 = simbad.query_object('NGC 2419') # carry out the query and store the result
ngc2419

Extract RA and DEC from this table and convert them to an `astropy` coordinate object:

In [ ]:
ngc2419_pos = c.SkyCoord(ngc2419['RA'],ngc2419['DEC'],unit=(u.hourangle,u.degree))
print(ngc2419_pos)

We'll start by getting an image of cluster, so we know what we're dealing with. In this example we'll get an image in the SDSS `g` band (around the middle of the visible spectrum). For this we use a SkyView query (we could have used SDSS.get_images, but SkyView can join together multiple SDSS fields into a single larger image - this would be much harder to do if we got the individual images from SDSS ourselves).

In [ ]:
import astroquery.skyview
skyview = astroquery.skyview.SkyView()
# Note: skyview.list_surveys() shows the various image sources available.

pl.figure(figsize=(8,8))

# This takes a while
im = skyview.get_images(ngc2419_pos,survey=['SDSSg'],pixels='600',radius=20.0*u.arcmin)
pl.imshow(im[0][0].data,cmap='magma',origin='lower',vmax=0.5);

The cluster is the bright blob in the centre of the image. Notice the very bright star to the right (west) of the cluster, at coordinates (400,300) in this image. This is a relatively nearby star that appears almost as bright as the cluster (which comprises millions of stars, but is much further away).

It would be better to have numbers in degrees on the axes (rather than the index of the pixels). We also want to make sure the sky projection is correct (after all, these pixels we're looking at are projections onto a spherical surface -- the sky is not flat). 

The difficult business of projecting stuff onto the sphere of the sky is dealt with by a standard called WCS, which stands for World Coordinate System. If you're an astronomer, you can your supervisor to explain what WCS is and why it's important (if your supervisor is a theorist, you may get a blank look...). For this tutorial we won't worry about this. The basic idea is that most astronomical images (that come from professional observatories) carry around WCS information in their FITS headers that can be used to project their CCD pixels properly onto your screen with accurate coordinate axes (accounting both for where the telescope is pointing and for the optical distortion between the sky and the CCD plane). 

We can just use the `astropy.wcs` module to get a WCS definition from the header of the image we downloaded in the previous cell, and use that to define all the projections we'll make on the subsequent plot.

If we want to plot a point with a given (RA,DEC) on top of our image, we first have to use the WCS projection to convert that (RA,DEC) to an (X,Y) position on our image. The WCS module in `astropy` has a function `all_world2pix` to do this (examples will follow below). For example, in the simplest common case, this would just do a tangent-plane projection of our spherical coordinate onto the plane of the image.

Before we do any of that, if you've never seen a FITS header before you can execute the following cell. The entries up to the first line starting COMMENT are the WCS portion of the header.

In [ ]:
im[0][0].header # Just to show you what a FITS header (with WCS keywords) looks like.

OK, here is some example code to use the WCS information when we make a plot of our image.

In [ ]:
import astropy
pl.figure(figsize=(8,8))

# Grab the WCS header from our image and use that to project everything else
h   = im[0][0]
wcs = astropy.wcs.WCS(h.header)

# This is where we tell matplotlib about the WCS projection we want.
pl.subplot(111,projection=wcs)

# Plot the image on the axes we set up with the WCS projection
pl.imshow(im[0][0].data,cmap='magma',origin='lower',vmax=0.5,alpha=0.6)

pl.grid(True,color='k',alpha=0.7,ls='--')

# Mark the cluster with a dot.
x,y = wcs.all_world2pix(ngc2419_pos.ra,ngc2419_pos.dec,0) 
pl.scatter(x,y,c='r',s=25)

# Draw a label for this point using annotate, a slightly complicated
# function from matplotlib.
pl.annotate('NGC 2419',(x,y),(x+40,y+90),color='k',fontsize=10,zorder=10,
            arrowprops=dict(arrowstyle="-", connectionstyle='arc'),
            bbox=dict(facecolor='white', edgecolor='k',boxstyle='round'));

# Store the axes object, we'll need it in a moment
ax = pl.gca()

# Plot a circle of radius 5 arcmin around the cluster. 

# From the astropy documentation on SphericalCircle:
# "This class is needed in cases where the user wants to add a circular patch
# to a celestial image, since otherwise the circle will be distorted, because 
# a fixed interval in longitude corresponds to a different angle on the sky 
# depending on the latitude."
from astropy.visualization.wcsaxes import SphericalCircle
my_circle = SphericalCircle((ngc2419_pos.ra,ngc2419_pos.dec),
                      5.0*u.arcmin, edgecolor='k', facecolor='None',
                      transform=ax.get_transform('icrs'))

# Put the circle onto the axes.
ax.add_patch(my_circle)

# Finally format the plot:

# See here for how to format the tick marks
# http://wcsaxes.readthedocs.io/en/latest/ticks_labels_grid.html
# In our case:
lon, lat = ax.coords
lon.set_major_formatter('d.dd')
lat.set_major_formatter('d.dd') # d.dd means degrees with 2 decimal places

lon.set_ticks(spacing=5.0 * u.arcmin)
lat.set_ticks(spacing=5.0 * u.arcmin)

pl.ylim(0,600)
pl.xlim(0,600)

pl.xlabel(r'$\alpha$ $(\degree)$',fontsize=12)
pl.ylabel(r'$\delta$ $(\degree)$',fontsize=12);

Now we'll issue another query to get data on each of the sources in this image from SDSS. We'll download all the entries in the photometric detection table (`photoobj`) within 10 arcminutes of the position of the cluster. We'll ask for position (`ra` and `dec`), the `g` and `r` band PSF magnitudes (the appropriate magnitudes to use for for stars), the SDSS source class (`type`) and the standard set of SDSS identifiers (`objid`, `run`, `rerun`, `camcol` and `field`) which are not used in this example, but which would be useful if we wanted to carry out further queries, e.g. for spectra or additional properties). 

In [ ]:
import astroquery.sdss
sdss = astroquery.sdss.SDSS()
res  = sdss.query_region(ngc2419_pos,radius=10*u.arcmin,
                       photoobj_fields=['ra','dec','objid','run','rerun','camcol','field',
                                        'psfMag_g', 'psfMag_r','extinction_g','extinction_r','type'])

In [ ]:
res

Make a quick dotplot of the sources to check we have something reasonable:

In [ ]:
pl.figure(figsize=(6,6))
pl.scatter(res['ra'],res['dec'],s=1,edgecolor='None',c='k');

This looks OK, but note that there is an 'empty' ring around the cluster position (the centre of the field) with no points in it. This is because the SDSS photometry pipeline fails in very crowded fields. 

The second empty region to the 'left' of the cluster is due to the very bright star there, which also limits the SDSS pipeline for detecting sources. Notice that the scatter plot is flipped left-to-right compared to the image we made above. This is because, by convention, astronomers make plots with right ascension (the x axis in these figures) increasing from right to left, not left to right. This is taken into account automatically by the WCS information we used to plot the image, but we didn't use that when we made the scatter plot.

***Exercise:***  In the cell below, make a version of the WCS image plot that overplots the catalog points as dots. Beware that you may have to convert the RA and DEC to astropy Angle objects.

### Finding points within a spherical angular distance

Notwithstanding the missing sources near the cluster, let's make a (rough!) colour magnitude diagram. This is a plot of the color (ratio of brightness in two different wavelengths) of a star, conventionally on the x axis, against its apparent brightness (in one of those two bands, or another). The physics of stellar evolution constrains stars to travel along well-defined tracks in this space as they age. 

In [ ]:
pl.figure(figsize=(8,8))

# First make a colour magnitude diagram of all points
rmag = res['psfMag_r'] - res['extinction_r']
gmr  = (res['psfMag_g']-res['extinction_g'])-(res['psfMag_r']-res['extinction_r'])
pl.scatter(gmr,rmag,s=4,c='k',edgecolor='None',label='All sources')
pl.xlim(-1,2)
pl.ylim(23,14)
pl.xlabel('$g-r$',fontsize=12)
pl.ylabel('$r$',fontsize=12)
pl.legend(loc='upper left',frameon=True,fontsize=12,markerscale=2);

This CMD contains every source in the image -- as well as stars in the cluster, these will included a large number of  much closer stars and some bright background galaxies. Because stars in the cluster are all at the same distance, their CMD track ('isochrone' in the jargon) will stand out as a coherent feature in plots like this, whereas foreground stars and background galaxies will give rise to much broader overdensities of points.

The next cell uses the `type` column in the data to separate stars from galaxies (or rather, to show how the SDSS pipeline has separated stars from galaxies).

In [ ]:
pl.figure(figsize=(8,8))

# First make a colour magnitude diagram of all points
rmag = res['psfMag_r']-res['extinction_r']
gmr  = (res['psfMag_g']-res['extinction_g'])-(res['psfMag_r']-res['extinction_r'])

pl.scatter(gmr,rmag,s=4,c='k',edgecolor='None',label='All sources')
pl.xlim(-1,2)
pl.ylim(23,14)
pl.xlabel('$g-r$',fontsize=12)
pl.ylabel('$r$',fontsize=12)

# Overplot objects classified as stars (SDSS type = 6)
# (read the SDSS data model for more details about this type)
is_sdss_star = res['type'] == 6
pl.scatter(gmr[is_sdss_star],rmag[is_sdss_star],s=4,c='cyan',edgecolor='None',label='STAR class');

pl.legend(loc='upper left',frameon=True,fontsize=12,markerscale=2);

One quick way to make the cluster CMD sequence stand out is to use the fact that it is very dense, so most sources near the cluster are very likely to be stars in the cluster. We can isolate points from the SDSS catalogue that are within some angular distance (in this case we'll take 5 arcmin) of the centre of the cluster. 

Because we're working with a spherical geometry, it wouldn't be correct to use a (Euclidean) KDTree method. Instead, Astropy coordinate objects have a useful `separation` method that takes a list of coordinates and calculates their *angular* separation to that point. There are several associated routines to match catalogues (two big arrays of coordinates) by position, but I haven't included examples of those (these still use a KDTree behind the scenes, but they account for the spherical geometry).

In [ ]:
pl.figure(figsize=(8,8))

# Only plot objects classified as stars (SDSS type = 6)
# (read the SDSS data model for more details about this type)
is_sdss_star = res['type'] == 6

# First make a colour magnitude diagram of all points
rmag =  res['psfMag_r']-res['extinction_r']
gmr  = (res['psfMag_g']-res['extinction_g']) - (res['psfMag_r']-res['extinction_r'])

pl.scatter(gmr[is_sdss_star],rmag[is_sdss_star],s=4,c='lightgrey',edgecolor='None',label='All sources')
pl.xlim(-1,2)
pl.ylim(23,14)
pl.xlabel('$g-r$',fontsize=12)
pl.ylabel('$r$',fontsize=12)

# Get the ra and dec or all points in the SDSS data we downloaded
photo_points  = c.SkyCoord(res['ra'],res['dec'],unit=u.degree)

# FIND OBJECTS WITHIN 5 arcmin OF THE CLUSTER 
nearby        = ngc2419_pos.separation(photo_points).arcmin < 5
nearby_points = np.where(nearby & is_sdss_star)[0]

# Plot the colour and magnitude of these points in red
pl.scatter(gmr[nearby_points],rmag[nearby_points],s=20,c='red',edgecolor='None',label='NGC 2419 ($<5$")')
pl.legend(loc='upper left',frameon=True,fontsize=12,markerscale=2);

The red points clearly show the line of the globular cluster's giant branch (from `(0.5,20)` to `(1.5,17)` and so-called *horizontal branch* at `(-0.2,20.5)`. The spike of grey points on the righthand side of the plot (around `g-r = 1.5`) is the metal-rich main sequence of Milky Way disk stars.

We could proceed from here to fit a model (*isochrone*) to this sequence and derive the distance to the cluster. We can do a very rough estimate of that by eye, knowing that horizontal branch stars around `g-r = -0.25` have an absolute magnitude of `Mr=0.7`. In the diagram above this corresponds (by eye!) to an apparent magnitude of about `20.5`, so the distance modulus is:

In [ ]:
dmod = 20.5 - 0.7
dmod

Hence the distance is approximately:

In [ ]:
distance = 10**(1+(dmod/5.0))*u.parsec
print('%4.1f kpc'%(distance.to(u.kiloparsec).value))

This isn't such a bad estimate; the literature value for the distance modulus is roughly 19.8.

## Cosmology

[This](http://www.astropy.org/astropy-tutorials/edshift_plot.html) is a good introduction to the `astropy.cosmology` subpackage, for working with relations between redshift, distance, age, angular diameter, luminosity and so on, for different cosmological parameters.

***Exercise***: use the cosmology tools in astropy to plot the relation between redshift and the age of the Universe for the first-year Planck cosmological parameters. Use the same tools to find the lookback time (t=0 at the present day) to redshift 3, the redshift at which the age of the universe is 4 Gyr, and the redshifts at which an angular scale of 1 arcsecond corresponds to a physical scale of 7 kpc.

## Planning observations

`PyEphem` is a useful package for planning observations according to the rising and setting times of astronomical objects (including the Sun and Moon) at specific geographical locations. Documentation and an extensive tutorial here: http://rhodesmill.org/pyephem/

In [ ]:
import ephem
import datetime

The first step in most calculations is to specify an observer. In this example, we'll set up an observer in Durham.

In [ ]:
durham = ephem.Observer()
durham.lat = np.deg2rad(54.7761) # deg
durham.lon = np.deg2rad(-1.5733) # deg
durham.elevation = 50 # m above sea level
print(durham.lat)
print(durham.lon)

Note that PyEphem assumes all angles passed as floats are in radians. We could have given the lon and lat as strings `'54:46:34.0'` and `'-1:34:23.9'` instead.

What time is it?

In [ ]:
print(ephem.now())

Where in the sky are the Sun and Moon right now?

In [ ]:
sun = ephem.Sun()
sun.compute(durham)

moon = ephem.Moon()
moon.compute(durham)

print("Sun  -- Alt: %12s   Az: %12s" % (sun.alt,  sun.az))
print("Moon -- Alt: %12s   Az: %12s" % (moon.alt, moon.az))

The functions to compute the next rising and setting are almost self-explanatory:

In [ ]:
print(durham.next_setting(sun))

In [ ]:
print(durham.next_rising(moon))

These calculations assume the horizon is at zero degrees altitude. In practice there are many definitions of 'rising' and 'setting'. The astronomical definition requires the Sun to be 18 degrees below the horizon. Also, we care about refraction at high airmass and we might even care about the difference between the centre of the Sun and its edge -- all of these can be dealt with by PyEphem and most of them are explained in the documentation.

The following simple example plots a chart showing the altitude of the Moon and the Sun as a function of time since midnight. First, we make a date object representing the last midnight, using Python's `datetime`.

In [ ]:
now = datetime.datetime.now()
midnight_today = datetime.datetime(year=now.year,month=now.month,day=now.day)
print(midnight_today)

Now we create an observer at Durham whose local time we'll vary.

In [ ]:
observer      = durham.copy()
observer.date = midnight_today

Now compute the altitude of the Sun (in degrees) for every 30 min between the last midnight and the next. Times in PyEphem are stored as fractional days from some epoch (read the documentation) which means we can add values to them directly provided we use the same units (days). Some constants are included in `ephem` to help with these calculations, for example:

In [ ]:
print(ephem.minute) # This is 1 minute in days
print(1.0/(24.0*60.0))

In [ ]:
# Grid of times
times   = ephem.minute*np.arange(0,24.0*60.0,30)
dates   = [temp_durham.date + t for t in times]

sun_alt  = list()
moon_alt = list()
for new_date in dates:
    observer.date = new_date
    sun.compute(observer)
    moon.compute(observer)
    
    # Store altitude in degrees
    sun_alt.append(np.rad2deg(sun.alt))
    moon_alt.append(np.rad2deg(moon.alt))

# Reset the date
observer.date = midnight_today

Now we can plot the results, including lines showing sunrise and sunset.

In [ ]:
f = pl.figure(figsize=(10,4))
pl.scatter((np.array(dates)-observer.date)/ephem.hour, sun_alt, 
           s=200,edgecolor='k',facecolor='y',alpha=0.5,label='Sun')

pl.scatter((np.array(dates)-observer.date)/ephem.hour, moon_alt, 
           s=100,edgecolor='k',facecolor='c',alpha=0.5,label='Moon')

pl.axhline(0,c='k',ls='--')
pl.axvline((observer.next_rising(sun)-observer.date)/ephem.hour,c='b')
pl.axvline((observer.next_setting(sun)-observer.date)/ephem.hour,c='r')

ax = pl.gca()
ax.set_xticks(np.arange(0,24))
pl.xlabel('Time since midnight (hours)')
pl.ylabel('Altitude (degrees)')
pl.title('Sun and Moon altitude at Durham from %s'%(observer.date))
pl.legend();

## Healpix and Healpy

The [`healpy` package](https://healpy.readthedocs.io/en/latest/) allows python to interact with [NASA's `HEALPIX` software](http://healpix.jpl.nasa.gov/). HEALPIX is a way of working with a clever equal-area pixelisation of the sphere, which is handy for anything involving statistics (or plots...) of distributions of points on the sky. It was invented mainly for work with CMB maps, but it is more widely useful for anything that's basically a histogram in spherical coordinates. Read the HEALPIX page for more details.

Obviously, `HEALPIX` needs to be installed to use `healpy`. `pip install healpy` claims will install a copy of `HEALPIX` for you (see the installation instructions at the `healpy` link above). I have had mixed results with this in the past. If you're installing Healpy on your own machine, I recommend using `conda`; see the section 'Notes on Conda Channels' at the end of this notebook.

Once it's installed, you can look at the tutorial on the `healpy` page (link above). The following example shows a common use case, which is just to use `HEALPIX` as a way to make a histogram of points on the sky, exploiting the fact that each of the `HEALPIX` pixels has the same area. To do this, we first have to assign points to HEALPIX pixels (using `ang2pix`), then make a histogram of the pixel numbers (using `numpy`) and finally plot that histogram as a `HEALPIX` map.

(`healpy` is slow to catch up with some of the more recent advances in matplotlib, so its plots look a bit old-fashioned sometimes...)

## What next?

You could try the official Astropy tutorials: http://www.astropy.org/astropy-tutorials/

In [ ]:
import healpy as hp

# Read the Sagittarius stream data from last week
from astropy.table import Table
d = Table(np.genfromtxt('%s/all_particles.txt'%(datadir),names=True,delimiter=','))

# Galactic coordinates of stream stars
l,b = d['l'].data, d['b'].data

# These are ranges of galactic coordinates from the file
print('Galactic coordinate ranges (degrees):')
print(l.min(), l.max())
print(b.min(), b.max())

# Healpix has some particular conventions for angular coordinates
# See http://healpix.jpl.nasa.gov/html/intronode6.htm
print('HEALPIX coordinate ranges (radians/pi):')

# theta is the healpix convention for *latitude*: [0,pi]
theta = (b+90)*np.pi/180.0 
print(theta.min()/np.pi,theta.max()/np.pi)
# phi is the healpix convention for *longitude*: [0,2*pi)
phi = l*np.pi/180.0
print(phi.min()/np.pi,phi.max()/np.pi)

# The resolution of healpix maps is determined by 'nside', 
# which must be a power of 2 (up to 2**30)
nside = 2**6

# If we want to know what the resolution is in square degrees,
# healpy can tell us the pixel area:
pixel_area = hp.nside2pixarea(nside,degrees=True)
print('Area of nside={} pixels is {} sq. deg.'.format(nside,pixel_area))

# First step: find the Healpix pixel for each star
pix    = hp.ang2pix(nside,theta,phi,nest=True)

npix   = hp.nside2npix(nside)   # Number of pixels for this value of nside
bins   = np.arange(0,npix+1)    # Have to have exactly npix bins!

# Make the histogram of pixel numbers
h, _ = np.histogram(pix,bins=bins)  

# Convert this to number per square degree
h = h/pixel_area

# Side note: "_" is a python convention; if something is assigned 
# to "_" it usually means you don't care about that thing; it's an 
# alternative to calling the variable 'junk' or something similar.
# There is nothing deeply special about the name "_". Here we assign
# the bins returned by histogram to _, because we already have them
# in the variable 'bins'.

# Plot a Mollweide projection of the density of stars
cmap = pl.get_cmap('viridis')
cmap.set_under('w') 
hp.mollview(h,rot=(90.0,0,0),nest=True,coord='G',cmap=cmap,
            min=0,max=25)

# Add lines
hp.graticule(c='lightgrey')
hp.visufunc.projtext(np.pi/2.0,0,'Galactic centre',color='w')
hp.visufunc.projtext(np.pi/2.0,np.pi,'Anticentre',color='w')

# Plot an 'orthographic' view
cmap = pl.get_cmap('magma_r')
cmap.set_under('w') 
hp.orthview(h,nest=True,cmap=cmap)
hp.graticule(c='k')

# Plot a Mollweide projection again, with graticule lines showing
# RA and DEC coordinates
hp.mollview(h,nest=True,coord='G',cmap='viridis',title='Galactic coordinates, equatorial graticule')
hp.graticule(coord='E',c='lightgrey')

### Notes about `conda` channels

`healpy` is not as straightforward to install with `conda` as the packages we've met so far, because it's not in the default location `conda` looks for packages on the web when you type something like `conda install healpy` (at least, not for the mac -- it might be for linux). The same was true for the `astroquery` package, but in that case we jumped straight to the solution (use the option `-c astropy` to select the `astropy` channel). What if we know there's a package called `healpy` somewhere, but we don't know what `conda` channel it's in?

This is quite common, so it's worth mentioning how to solve it.  `conda` provides a chain of clues that get you to the right place in the end. The following (non-executable) cells illustrate how this worked on my system:

If we follow that instruction:

This is a list of packages from non-default locations ('channels'). Some of these look promising -- `OpenAstronmy` sounds good, `fermipy` might sound familiar, as might `lsst`. Let's go with `OpenAstronomy` because it has `Astronomy` in the name (also it's what the `healpy` author recommends, if you read the installation instructions for that package at the link above). The next step is given at the top of the above output: `Run 'anaconda show <USER/PACKAGE>' to get more details`.

Finally! The last line shows you how to actually install this package with `conda`.

# End of notebook